# Part of Speech Tagging with HMMs and Viterbi decoding

Hidden Markov Models are often used to represent sequence data in NLP problems.  In this notebook, we use HMMs to model parts of speech of word sequences.

A HMM models a system that consists of two parts:
1.  A sequence of unobservable "states", $y_i$.  At each step, the system is in one of these.
2.  A sequence of observable outputs (sometimes called "emissions"), $x_i$.  These outputs are produced as a result of the state the system is in at that step.

<img src="hmm.png">

Recall that the "Markov" part of the HMMs acronym means that we are making some simplifying assumptions when modelling in this way:
1.  The set of states are available for the next step (and with what distribution) depends only on the state at the current step.
2.  The output generated at each step depends only on the state at that step.

You can imagine that each node in the image of the HMM/graphical model above contains a conditional probability table that enumerates the probability distributions for the node given its "parent".

In this notebook:
1.  States correspond to parts of speech
2.  Observable outputs are the words in the sentence.

This aligns well with the HMM model definition above:  in any given sentence it's trivial to observe the words but the parts of speech are hidden.  This is called a "generative model" as the natural form of it allows you to run it forwards and "simulate" state transitions and corresponding emissions.

This is similar to the language modelling we did earlier in the course (Weeks 2 and 4).  Just like in those cases, the reason we are doing this isn't to generate text.  Instead, we're using the model to "score" candidates.  In this case, these candidates are part of speech tag assignments.

We'll describe the application in more detail when we get to the code below.  First, let's set things up by importing a few useful libraries.

In [1]:
from collections import Counter
from collections import defaultdict
import nltk
from nltk.corpus import treebank
import numpy as np

The NLTK library only has ~10% of the Penn Treebank P.O.S. set.  It contains ~100K tokens consisting of ~12K types.  

The treebank is available as part of the LDC Corpora access that you get by being a UC Berkeley student.  We use the NLTK subset here for convenience.

These counts are big, but they aren't particularly huge.  As we learned in Week 2, the more data you have, the better estimates of probabilities you get.  Even with the full treebank, the smoothing techniques you learned about come in handy to deal with OOV words, rare transitions - or, more generally, sparsity.

In this next cell, we dump the first 10 tokens and their part of speech tags to the output.  A [full list](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) of these tags and their definitions was developed as part of the treebank project.  (There is no authoritative set of tags that everyone uses!  The Penn project had detailed specifications on how to tag words and parse sentences - something we'll learn about in a couple of weeks.)

In the rest of this document, we refer to the tokens in the sentences as "words" and their part of speech tags as "tags".

In [2]:
print 'The treebank has %d tokens with %d types.' % (len(treebank.tagged_words()), len(set(treebank.words())))
treebank.tagged_words()[0:10]

The treebank has 100676 tokens with 12408 types.


[(u'Pierre', u'NNP'),
 (u'Vinken', u'NNP'),
 (u',', u','),
 (u'61', u'CD'),
 (u'years', u'NNS'),
 (u'old', u'JJ'),
 (u',', u','),
 (u'will', u'MD'),
 (u'join', u'VB'),
 (u'the', u'DT')]

In [3]:
# This usage of zip, if you aren't familiar with it, turns a list of tuples
# "inside out":
# l = [(1, 2), (3, 4), (5, 6)]
# zip(*l) is [(1, 3, 5), (2, 4, 6)]
#
# In this case zip(*treebank.tagged_words()) results in
# [(word1, word2, word3), (tag1, tag2, tag3)]

# The sequence of part of speech tags corresponding with the sequence of
# words in the corpus.
tags = zip(*treebank.tagged_words())[1]

# A (unique'd) set of all the tags available.
tag_set = set(tags)

# A histogram of tags.
tag_count = Counter(tags)

# A histogram of (word, tag) tuples.
word_tag_count = Counter(treebank.tagged_words())

# A histogram of (tag_next, tag_current) tuples.
tag_tag_count = Counter(
    (y2, y1) for y1, y2 in zip(tags[0:len(tags) - 1], tags[1:]))

Let's take a look at the tags in the Penn dataset, along with their frequency.  A couple of these are pretty sparse.  The goal of this document is the clarity of Viterbi, not the performance of the algorithm, so we'll acknowledge that limitation and continue on anyways.

In [4]:
tag_count

Counter({u'#': 16,
         u'$': 724,
         u"''": 694,
         u',': 4886,
         u'-LRB-': 120,
         u'-NONE-': 6592,
         u'-RRB-': 126,
         u'.': 3874,
         u':': 563,
         u'CC': 2265,
         u'CD': 3546,
         u'DT': 8165,
         u'EX': 88,
         u'FW': 4,
         u'IN': 9857,
         u'JJ': 5834,
         u'JJR': 381,
         u'JJS': 182,
         u'LS': 13,
         u'MD': 927,
         u'NN': 13166,
         u'NNP': 9410,
         u'NNPS': 244,
         u'NNS': 6047,
         u'PDT': 27,
         u'POS': 824,
         u'PRP': 1716,
         u'PRP$': 766,
         u'RB': 2822,
         u'RBR': 136,
         u'RBS': 35,
         u'RP': 216,
         u'SYM': 1,
         u'TO': 2179,
         u'UH': 3,
         u'VB': 2554,
         u'VBD': 3043,
         u'VBG': 1460,
         u'VBN': 2134,
         u'VBP': 1321,
         u'VBZ': 2125,
         u'WDT': 445,
         u'WP': 241,
         u'WP$': 14,
         u'WRB': 178,
         u'``': 712

Normalize the histograms to get the probability table (Pyy) for the hidden (y) nodes as well as the table (Pxy) for the observations.  Table Pab represents the value P(a | b).  Its key is a tuple (a, b) and its value is the probability value.

In [5]:
Pxy = {(x, y): np.log(count) - np.log(tag_count[y])
       for (x, y), count in word_tag_count.iteritems()}

Pyy = {(y2, y1): np.log(count) - np.log(tag_count[y1])
       for (y2, y1), count in tag_tag_count.iteritems()}

Finally, we are at the viterbi algorithm implementation.

The comments for this block are mostly inline, but a few observations before we get started:
- To execute this code, you pass a list of words as sentence (e.g. 'hello world'.split(), not 'hello world')
- verbose outputs the $\delta$ table for inspection.

In [6]:
def viterbi(sentence, verbose=False):
    # How do we score out of vocabulary words?
    # Obviously we should do something far more sophisticated (see week 2).
    # The point of this notebook is clarity and so we just do some very
    # unprincipled smoothing so we can focus
    # on the rest of the implementation below.
    if verbose:
        EPS = -100
    else:
        EPS = -float('inf')
        
    # Delta is the Dynamic Programming table - or, more simply: the
    # result cache.
    # Its values are going to store the answer to the question:
    # "What is the best way to get to this word in the sentence where
    # that word is tagged with that POS tag?"
    # 
    # As a result, delta's keys are tuples of the form
    # (word_index_into_sentence, tag_of_word_at_index)
    # 
    # Its values are (highest_logp_to_get_here, previous_tag_in_that_best_scoring_sequence)
    #   We use the first half of the value tuple to track of the
    #       "score so far".
    #   We use the second half of the value tuple to track the
    #       previous step in that optimal path.
    delta = {}
    
    # Here, we prime the pump.  In lecture, we start with P(y1).
    # This works, however, we might get a better
    # estimate of the possible start states to a sentence if we use
    # the transition probabilities from a period
    # instead.  To that end, we pretend we have an extra hidden state
    # whose value is deterministically the tag
    # '.'.  Its score is 0.0 as that is log of a probability of 100%. 
    # We assign a epsilon probability to every other possible tag.
    for tag in tag_set:
        delta[(-1, tag)] = (EPS, '')
    delta[(-1, '.')] = (0.0, '')
    
    # Here is the main work of the Viterbi algorithm...
    # For each i=index, w=word in the sentence...
    for i, x in enumerate(sentence):
        # We consider and score every possible part of speech tag as
        # an assignment to word x...
        for y2 in tag_set:
            # We determine which path through the previous word tag
            # assignments is best when combined with
            # assigning the current word (x) this tag (y2).  This
            # answers the question: "if you force me to
            # assign tag y2 to word x, what is the best assignments to
            # the previous words to make the whole
            # collection as likely as possible?"
            # best_transition returns a tuple of
            # (score_up_to_hidden_state_with_assignment_y2, previous_word's_tag)
            best_transition = max(
                [(delta[(i - 1, y1)][0] + Pyy.get((y2, y1), EPS), y1)
                 for y1 in tag_set])
            
            # ... we then multiply in (add in log space) the
            # probability of seeing word x given state y2
            # and then set the entry in the delta table for the best
            # score and path should we assign tag y2 to word x.
            delta[(i, y2)] = (best_transition[0] + Pxy.get((x, y2), EPS), best_transition[1])

    # Look through the assignment options to the last word.  Recall
    # that the scores associated with those assignments are the score
    # of the optimal sequence of assignments up until that
    # point - in this case, the whole sentence.
    #
    # Pick the best scoring assignment.
    decoding_tail = max(
        [(delta[(len(sentence) - 1, tag)][0], tag)
         for tag in tags])
    
    # (Interlude...) If requested, output the delta table for inspection.
    if verbose:
        for i in xrange(len(sentence) + 1):
            for tag in sorted(tag_set):
                if delta[i-1, tag][0] > 2 * EPS:
                    print i - 1, tag, delta[i-1, tag]
            print '\n\n'
    # (...end interlude)

    # Decode the sequence of tags from the table by following the
    # "previous" links back through it, starting from the end.
    decoding = [decoding_tail[1]]
    for i in xrange(len(sentence)):
        decoding.append(delta[(len(sentence) - i - 1, decoding[-1])][1])
        
    # Walking backwards gets the tags out in the wrong order... reverse.
    return [tag for tag in reversed(decoding)][1:]

Take a sentence from the treebank and use Viterbi to decode.

(A friendly reminder that given the amount of data we have, it's pretty easy to go out of vocabulary... but let's try a few sentences anyways.  First we'll grab a longish one from the training set.)

In [7]:
words, tags = zip(*[tagged_word for tagged_word in treebank.tagged_words()][:18])
print ' '.join(words)
zip(tags, viterbi(words))

Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov. 29 .


[(u'NNP', u'NNP'),
 (u'NNP', u'NNP'),
 (u',', u','),
 (u'CD', u'CD'),
 (u'NNS', u'NNS'),
 (u'JJ', u'JJ'),
 (u',', u','),
 (u'MD', u'MD'),
 (u'VB', u'VB'),
 (u'DT', u'DT'),
 (u'NN', u'NN'),
 (u'IN', u'IN'),
 (u'DT', u'DT'),
 (u'JJ', u'JJ'),
 (u'NN', u'NN'),
 (u'NNP', u'NNP'),
 (u'CD', u'CD'),
 (u'.', u'.')]

In [8]:
viterbi('tea is tasty'.split())

[u'NN', u'VBZ', u'JJ']

In [9]:
viterbi('my computer is hot'.split())

[u'PRP$', u'NN', u'VBZ', u'JJ']

In [10]:
viterbi('give me a sentence'.split())

[u'VB', u'PRP', u'DT', u'NN']

In [11]:
viterbi('what was that'.split())

[u'WDT', u'VBD', u'IN']

In [12]:
viterbi('that car'.split())

[u'DT', u'NN']

In [13]:
viterbi('the food made by the man'.split())

[u'DT', u'NN', u'VBD', u'IN', u'DT', u'NN']

In [14]:
viterbi('the men walked'.split())

[u'DT', u'NNS', u'VB']

In [15]:
viterbi('that was weird'.split())

[u'IN', u'VBD', u'JJ']

In [16]:
viterbi('the car that went too fast'.split())

[u'DT', u'NN', u'WDT', u'VBD', u'RB', u'JJ']

In [17]:
viterbi('the car that went way too fast'.split())

[u'DT', u'NN', u'WDT', u'VBD', u'NN', u'RB', u'JJ']